In [17]:
import os
import json
import warnings
import optuna
import numpy as np
import pandas as pd
from io import StringIO
from pathlib import Path
from datetime import datetime

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
#from catboost import CatBoostRegressor, sum_models

In [20]:
height = [5, 5, 5, 5, 5, 5, 5, 3, 3, 1, 1, 1, 5, 5, 1, 1, 0]
angle = [181, 175, 180, 161, 208, 208, 172, 219, 151, 223, 131, 298, 249, 197, 127, 82, 0]


for i in range(1, 18):
    dataset_dir = os.getcwd() + f"/36_TrainingData/L{i}_Train.csv"
    print(dataset_dir)
    df = pd.read_csv(dataset_dir)

    # with additional data
    if i in [2, 4, 7, 8, 9, 10, 12]:
        addition_dir = os.getcwd() + f"/36_TrainingData_Additional_V2/L{i}_Train_2.csv"
        df2 = pd.read_csv(addition_dir)
        df = pd.concat([df, df2], axis = False)

    df['DateTime'] = pd.to_datetime(df['DateTime'])
    df['height'] = height[i-1]
    df['angle'] = angle[i-1]
    col = df.pop("Power(mW)")
    df["Power(mW)"] = col

    df.set_index('DateTime', inplace=True)
    df = df.resample('10T').mean()
    df = df.round(2)

    df.reset_index(inplace=True)
    # df.replace("", np.nan, inplace=True)
    df = df.dropna()

    df.to_csv(os.getcwd() + f"/TrainingData_avg/L{i}_Train.csv", index=False)

/Users/kase/Documents/113-1/AI Cup/BlaBlaBlazzz/36_TrainingData/L1_Train.csv


FileNotFoundError: [Errno 2] No such file or directory: '/Users/kase/Documents/113-1/AI Cup/BlaBlaBlazzz/36_TrainingData/L1_Train.csv'

In [18]:
target_dir = os.getcwd() + f"/upload.csv"
target = pd.read_csv(target_dir, encoding='utf8')['序號'].values

date_device = {}
for item in target:
    date = str(item)[:8]
    device = str(item)[12:]
    dd = date + device

    if dd not in date_device.keys():
        date_device[dd] = [int(device)]
        
data_ls = []

for date in list(date_device.keys()):
    # traverse all dataset to find target days
    for i in range(1, 18):
        data_dir = os.getcwd() + f"/TrainingData_avg/L{i}_Train.csv"
        data = pd.read_csv(data_dir)
        data['DateTime'] = pd.to_datetime(data['DateTime'])
        data_ls.append(data)  # store data

        date_ls = data['DateTime'].dt.strftime("%Y%m%d").values

        if date[:-2] in date_ls and i not in date_device[date]:
            date_device[date].append(i)

# write file        
with open("target_match_device.txt", 'w') as file:
    file.write(json.dumps(date_device))

date_device


FileNotFoundError: [Errno 2] No such file or directory: '/Users/kase/Documents/113-1/AI Cup/BlaBlaBlazzz/upload.csv'

In [19]:
def make_date_device(data_ls):
    target_dir = os.getcwd() + f"/sample_upload.csv"
    target = pd.read_csv(target_dir, encoding='utf8')['序號'].values

    date_device = {}
    for item in target:
        date = str(item)[:8]
        device = str(item)[12:]
        dd = date + device

        if dd not in date_device.keys():
            date_device[dd] = [int(device)]
    # data_ls = load_data()

    for date in list(date_device.keys()):
        year = date[:4]
        month = date[4:6]
        day = date[6:-2]

        start_time = pd.Timestamp(f"{year}-{month}-{day} 09:00:00")
        end_time = pd.Timestamp(f"{year}-{month}-{day} 16:50:00")
        # traverse all dataset to find target days
        for i in range(1, 18):
            data = data_ls[i-1]

            filtered_df = data[(data['DateTime'] >= start_time) & (data['DateTime'] <= end_time)]
            if len(filtered_df)>40:
                date_device[date].append(i)
    
    # write file        
    # with open("target_match_device.txt", 'w') as file:
    #     file.write(json.dumps(date_device))

    return date_device

date_device = make_date_device(data_ls)

NameError: name 'data_ls' is not defined

In [29]:
def feature_engineering(df):
    df = df.copy()
    # df['hour'] = df['DateTime'].dt.hour
    # df['minute'] = df['DateTime'].dt.minute
    # baseline_date = pd.to_datetime('2024-01-01')
    # df['days'] = (df['DateTime'] - baseline_date).dt.days  # distance to the baseline date(2024-01-01)
    # df['month'] = df['DateTime'].dt.month


    # df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    # df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)

    # df['minute_sin'] = np.sin(2 * np.pi * df['minute'] / 24)
    # df['minute_cos'] = np.cos(2 * np.pi * df['minute'] / 24)

    # df['lag_sunlight'] = df['Sunlight(Lux)'].shift(1)
    # df['lag_power'] = df['Power(mW)'].shift(1)

    # df.drop(columns=['DateTime', 'height', 'angle', 'LocationCode'], inplace=True)
    df.drop(columns=['idx', 'DateTime', 'LocationCode', 'height', 'angle', 'is_upper', 'Sunlight(Lux)'], inplace=True)
    return df

def load_data():
    data_ls = []
    for i in range(1, 18):
        # data_dir = os.getcwd() + f"/TrainingData_avg/L{i}_Train.csv"
        # data_dir = os.getcwd() + f"/data/L{i}_Train.csv"
        data_dir = f"data/L{i}_Train.csv"
        data = pd.read_csv(data_dir)
        data['DateTime'] = pd.to_datetime(data['DateTime'])
        data.dropna(axis=0, how='any', inplace=True)
        data_ls.append(data)  # store data
    
    return data_ls

def load_date_device_dict():
    with open(os.getcwd() + "/target_match_device.txt") as f:
        date_dict = f.read()
    
    return json.loads(date_dict)



def get_intersection_date(data, id1, id2, date_id):
    data1 = data[id1-1]
    data2 = data[id2-1]
    target = data1[data1['DateTime'].isin(data2['DateTime'])].reset_index(drop=True)  # assign divice1 to the target power
    target.rename(columns={'Power(mW)': 'Label'}, inplace=True)
    target = target['Label']
    match2 = data2[data2['DateTime'].isin(data1['DateTime'])].reset_index(drop=True)

    # target test data
    start_time = pd.to_datetime(f"{date_id}0900", format='%Y%m%d%H%M')
    # print(start_time)
    end_time = pd.to_datetime(f"{date_id}1650", format='%Y%m%d%H%M')
    # print(end_time)
    test_data = data2[(data2['DateTime'] >= start_time) & (data2['DateTime'] <= end_time)].reset_index(drop=True)

    match2 = feature_engineering(match2)
    test_data = feature_engineering(test_data)

    return match2, target, test_data


def split_train_val_test(df, label):
    X_train, X_val, y_train, y_val = train_test_split(df, label, test_size=0.2, random_state=42)
    return X_train, X_val, y_train, y_val

def normalize_features(X_train, X_val, X_test):
    min_max_scaler = preprocessing.MinMaxScaler()

    X_train_scaled = min_max_scaler.fit_transform(X_train)
    X_val_scaled = min_max_scaler.transform(X_val)
    X_test_scaled = min_max_scaler.transform(X_test)

    X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns).astype(float)
    X_val_scaled_df = pd.DataFrame(X_val_scaled, columns=X_val.columns).astype(float)
    X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns).astype(float)

    return X_train_scaled_df, X_val_scaled_df, X_test_scaled_df


def complete_target_time():
    target_time = []
    for hour in range(9, 17):
        for minute in range(0, 60, 10):
            if hour == 16 and minute > 50:
                break
            target_time.append((hour, minute))
    
    return target_time


def fillna(X_test):
    target_time = complete_target_time()
    time_df = pd.DataFrame(target_time, columns=["hour", "minute"])

    df = time_df.merge(
        X_test,
        on=["hour", "minute"],
        how="left"
    )
    df.fillna(method="ffill", inplace=True)
    df.fillna(method="bfill", inplace=True) 
    df = df[X_test.columns]

    return df

def get_avg_of_closest4(data, date_id, result):
    df = feature_engineering(data)
    target_time = complete_target_time()

    # days from 2024-01-01
    date = pd.to_datetime(f"{date_id[:4]}-{date_id[4:6]}-{date_id[6:]}")
    baseline_date = pd.to_datetime('2024-01-01')
    target_day = (date - baseline_date).days

    for (hour, minute) in target_time:
        time_data = df[(df['hour'] == hour) & (df['minute'] == minute)]
        date_diff = np.abs(time_data['day'] - target_day)
        closest4_days = time_data.iloc[np.argsort(date_diff)[:4]]
        
        average_power = closest4_days['Power(mW)'].mean()
        result.append(np.round(average_power, 2))
    
    return result


def train_model(X_train, y_train, X_val, y_val):
    # def objective(trial):
    #     # 定義要搜尋的參數範圍
    #     param = {
    #         "n_estimators": trial.suggest_int("n_estimators", 50, 500),
    #         "max_depth": trial.suggest_int("max_depth", 3, 12),
    #         "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
    #         "subsample": trial.suggest_float("subsample", 0.5, 1.0),
    #         "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
    #         "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
    #         "gamma": trial.suggest_float("gamma", 0, 5),
    #     }

    #     # 建立模型
    #     model = XGBRegressor(**param, random_state=42, verbose=0)
    #     model.fit(X_train, y_train)

    #     # 驗證集預測
    #     y_pred = model.predict(X_val)
    #     mse = mean_squared_error(y_val, y_pred)
        
    #     # 最小化 MSE
    #     return mse
    
    # study = optuna.create_study(direction="minimize")
    # study.optimize(objective, n_trials=20, timeout=3600)

    # params = study.best_params
    # model = XGBRegressor(**params, random_state=42, n_jobs=-1)
    model = XGBRegressor(n_estimators=100, random_state=42, verbose=0)

    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)

    mse = mean_squared_error(y_val, y_pred)
    mae = mean_absolute_error(y_val, y_pred)
    return model, mse, mae

def evaluate_metrics(model, X_val, y_val):
    y_pred = model.predict(X_val)
    
    mse = mean_squared_error(y_val, y_pred)
    mae = mean_absolute_error(y_val, y_pred)

    return mse, mae

def run():
    data_ls = load_data()
    date_device = load_date_device_dict()
    total_mse = 0
    total_mae = 0
    
    result = []
    model_cache = {}

    # gg = {"2024011701": [1, 8, 17], "2024011901": [1, 2, 8, 17], "2024012502": [2, 1, 8, 17]}

    for idx, (date_id, device_ls) in enumerate(date_device.items()):
        date_id = date_id[:-2]
        best_mse = float('inf')
        best_mae = float('inf') #None
        best_model = None
        best_device = None
        
        sum_results = None
        model_weights = []

        # no data found in other device, take average of 4 closest days
        if len(device_ls) == 1:
            data = data_ls[device_ls[0]]  # data of target device
            result = get_avg_of_closest4(data, date_id, result)

            print("current problem: {}/200 | No device, fill with average power in 4 days".format(idx+1))

        else:
            model_len = len(device_ls[1:])
            for dev_idx, dev in enumerate(device_ls[1:]):
                train_data, label_data, X_test = get_intersection_date(data_ls, device_ls[0], dev, date_id)
                X_train, X_val, y_train, y_val = split_train_val_test(train_data, label_data)
                if X_test.shape[0] < 48:
                    X_test = fillna(X_test)
                # output[idx].append(48 - X_test.shape[0])

                # X_train, X_val, X_test = normalize_features(X_train, X_val, X_test)
                
                if (device_ls[0], dev) not in list(model_cache.keys()):
                    epoch_model, epoch_mse, epoch_mae = train_model(X_train, y_train, X_val, y_val)
                    model_cache[(device_ls[0], dev)] = [epoch_model, epoch_mse, epoch_mae]  # store pair model in cache
                else:
                    epoch_model, epoch_mse, epoch_mae = model_cache[(device_ls[0], dev)]
                
                model_weights.append(1 / epoch_mae)

                if epoch_mae < best_mae:
                    best_model = epoch_model
                    best_mse = epoch_mse
                    best_mae = epoch_mae
                    best_device = dev
                
                # aggregate test data
                weighted_prediction = epoch_model.predict(X_test) * (1/epoch_mae)
                sum_results = weighted_prediction if sum_results is None else sum_results + weighted_prediction
            
            
            total_mse += best_mse
            total_mae += best_mae
            
            print("current problem: {}/200 | Best MSE: {} | Best MAE: {} in Device {}".format(idx+1, best_mse, best_mae, best_device))
            
            # epoch_result = np.round(sum_model.predict(X_test), 2)

            epoch_result = np.round(sum_results / sum(model_weights), 2)
            result.extend(epoch_result)
        # display(X_test)

    print("All questions finished, Total MSE: {} | Total MAE: {}".format(total_mse, total_mae))
    

    return result



result = run()
# with open('result.txt', 'w') as f:
#     for line in result:
#         f.write(f"{line}\n")

    
    

/Users/kase/global_aicup/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [19:45:31] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/kase/global_aicup/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [19:45:31] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/kase/global_aicup/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [19:45:31] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/kase/global_aicup/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [19:45:31] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/kase/global_aicup/lib/python3.12/

current problem: 1/200 | Best MSE: 24846.470711239173 | Best MAE: 74.12615052634098 in Device 8
current problem: 2/200 | Best MSE: 2701.328494231145 | Best MAE: 17.86100926401878 in Device 2


/Users/kase/global_aicup/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [19:45:31] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/kase/global_aicup/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [19:45:32] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/kase/global_aicup/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [19:45:32] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/kase/global_aicup/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [19:45:32] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


current problem: 3/200 | Best MSE: 2701.328494231145 | Best MAE: 17.86100926401878 in Device 2
current problem: 4/200 | Best MSE: 9103.983479637409 | Best MAE: 39.4614941334826 in Device 14


/Users/kase/global_aicup/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [19:45:32] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/kase/global_aicup/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [19:45:32] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/kase/global_aicup/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [19:45:32] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/kase/global_aicup/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [19:45:32] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/kase/global_aicup/lib/python3.12/

...

current problem: 153/200 | Best MSE: 4333.284355345536 | Best MAE: 17.994901003924223 in Device 6
current problem: 154/200 | Best MSE: 4333.284355345536 | Best MAE: 17.994901003924223 in Device 6
current problem: 155/200 | Best MSE: 4333.284355345536 | Best MAE: 17.994901003924223 in Device 6
current problem: 156/200 | Best MSE: 4333.284355345536 | Best MAE: 17.994901003924223 in Device 6
current problem: 157/200 | No device, fill with average power in 4 days
current problem: 158/200 | No device, fill with average power in 4 days
current problem: 159/200 | Best MSE: 9968.576299102171 | Best MAE: 32.24541649764102 in Device 13
current problem: 160/200 | Best MSE: 5715.390524368922 | Best MAE: 26.31490111751901 in Device 3
current problem: 161/200 | Best MSE: 5715.390524368922 | Best MAE: 26.31490111751901 in Device 3
current problem: 162/200 | Best MSE: 11175.233001292869 | Best MAE: 32.130783600552554 in Device 6
current problem: 163/200 | Best MSE: 5715.390524368922 | Best MAE: 26.314

/var/folders/rr/xndz293x5zd33pfpcc2n9m600000gn/T/ipykernel_12957/1461234403.py:103: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill", inplace=True)
/var/folders/rr/xndz293x5zd33pfpcc2n9m600000gn/T/ipykernel_12957/1461234403.py:104: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="bfill", inplace=True)
/var/folders/rr/xndz293x5zd33pfpcc2n9m600000gn/T/ipykernel_12957/1461234403.py:103: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill", inplace=True)
/var/folders/rr/xndz293x5zd33pfpcc2n9m600000gn/T/ipykernel_12957/1461234403.py:104: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() ins

current problem: 170/200 | Best MSE: 19396.896644630626 | Best MAE: 68.61354950244443 in Device 7
current problem: 171/200 | Best MSE: 13032.602622094912 | Best MAE: 54.054948605953506 in Device 16
current problem: 172/200 | Best MSE: 13032.602622094912 | Best MAE: 54.054948605953506 in Device 16
current problem: 173/200 | Best MSE: 24846.470711239173 | Best MAE: 74.12615052634098 in Device 8
current problem: 174/200 | Best MSE: 36819.26897890424 | Best MAE: 94.4007960406295 in Device 17
current problem: 175/200 | Best MSE: 11469.409524950226 | Best MAE: 43.80634464541801 in Device 13
current problem: 176/200 | Best MSE: 1879.0803277996854 | Best MAE: 17.56620536946854 in Device 4
current problem: 177/200 | Best MSE: 1879.0803277996854 | Best MAE: 17.56620536946854 in Device 4
current problem: 178/200 | Best MSE: 1879.0803277996854 | Best MAE: 17.56620536946854 in Device 4
current problem: 179/200 | Best MSE: 1227.2304658620008 | Best MAE: 16.528948638578246 in Device 7
current problem

/var/folders/rr/xndz293x5zd33pfpcc2n9m600000gn/T/ipykernel_12957/1461234403.py:103: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill", inplace=True)
/var/folders/rr/xndz293x5zd33pfpcc2n9m600000gn/T/ipykernel_12957/1461234403.py:104: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="bfill", inplace=True)
/var/folders/rr/xndz293x5zd33pfpcc2n9m600000gn/T/ipykernel_12957/1461234403.py:103: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill", inplace=True)
/var/folders/rr/xndz293x5zd33pfpcc2n9m600000gn/T/ipykernel_12957/1461234403.py:104: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() ins

current problem: 182/200 | Best MSE: 1227.2304658620008 | Best MAE: 16.528948638578246 in Device 7
current problem: 183/200 | Best MSE: 14753.534237454624 | Best MAE: 57.05648350473877 in Device 10
current problem: 184/200 | Best MSE: 3373.616743223792 | Best MAE: 25.04844606544215 in Device 7
current problem: 185/200 | Best MSE: 16790.327064367382 | Best MAE: 60.739903974030234 in Device 10
current problem: 186/200 | Best MSE: 32409.578568948295 | Best MAE: 89.88084576102919 in Device 12
current problem: 187/200 | Best MSE: 6123.298342953594 | Best MAE: 32.88646868359875 in Device 4
current problem: 188/200 | Best MSE: 11446.280495098586 | Best MAE: 30.384807156440928 in Device 7
current problem: 189/200 | Best MSE: 6821.271672482247 | Best MAE: 25.965181250236444 in Device 4
current problem: 190/200 | No device, fill with average power in 4 days
current problem: 191/200 | Best MSE: 6821.271672482247 | Best MAE: 25.965181250236444 in Device 4
current problem: 192/200 | Best MSE: 11446

In [34]:
target_dir = os.getcwd() + f"/results/upload.csv"
upload = pd.read_csv(target_dir)

# result = [0 if r<0 else r for r in result]
result = np.round(result, 2)
upload['答案'] = result
upload.to_csv(os.getcwd() + "/results/upload_v5.csv", index=False)

In [32]:

result

[196.65,
 165.44,
 197.22,
 281.27,
 217.04,
 223.88,
 195.82,
 344.6,
 614.53,
 309.89,
 120.76,
 248.55,
 362.82,
 187.2,
 217.18,
 172.27,
 167.06,
 196.51,
 144.28,
 81.6,
 154.84,
 279.6,
 558.72,
 506.18,
 331.34,
 340.41,
 360.13,
 616.59,
 584.96,
 573.21,
 391.54,
 703.03,
 516.56,
 339.34,
 513.55,
 693.68,
 553.73,
 181.5,
 128.11,
 286.01,
 49.76,
 58.96,
 63.79,
 22.17,
 38.7,
 59.82,
 18.68,
 -9.77,
 526.66,
 944.67,
 810.99,
 691.91,
 674.75,
 725.5,
 847.1,
 859.62,
 849.27,
 1171.4,
 1181.74,
 1209.43,
 1168.82,
 1268.64,
 1053.22,
 1098.02,
 1172.84,
 1075.67,
 1162.55,
 874.24,
 789.65,
 820.93,
 775.64,
 1250.5,
 974.54,
 616.96,
 886.05,
 532.56,
 517.07,
 464.46,
 844.17,
 533.75,
 837.38,
 444.93,
 744.88,
 188.26,
 176.3,
 194.69,
 108.62,
 51.73,
 44.35,
 152.23,
 199.72,
 221.05,
 50.61,
 43.74,
 15.34,
 19.46,
 118.53,
 125.98,
 194.1,
 73.16,
 70.86,
 191.19,
 210.79,
 247.63,
 257.22,
 248.59,
 236.98,
 131.01,
 134.49,
 145.47,
 192.43,
 218.1,
 168.96,
 2

In [24]:
d = load_date_device_dict()
data_ls = load_data()
display(data_ls[0])
intersection, target, X_test = get_intersection_date(data_ls, 1, 8, '20240117')

X_train, X_val, y_train, y_val = split_train_val_test(intersection, target)
# X_train, X_val, X_test = normalize_features(X_train, X_val, X_test)
X_train

,idx,DateTime,LocationCode,WindSpeed(m/s),Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),height,angle,Power(mW),year,month,day,hour,minute,is_upper,SunlightPred
0,0,2024-01-01 06:30:00,1.0,0.0,1016.54,17.50,86.78,50.09,5.0,181.0,0.00,2024,1,1,6,30,0,50.09
1,1,2024-01-01 06:40:00,1.0,0.0,1016.59,17.49,86.80,163.25,5.0,181.0,0.02,2024,1,1,6,40,0,163.25
2,2,2024-01-01 06:50:00,1.0,0.0,1016.66,17.50,86.95,326.83,5.0,181.0,0.05,2024,1,1,6,50,0,326.83
3,3,2024-01-01 07:00:00,1.0,0.0,1016.73,17.50,86.99,470.83,5.0,181.0,0.08,2024,1,1,7,0,0,470.83
4,4,2024-01-01 07:10:00,1.0,0.0,1016.77,17.57,86.97,659.67,5.0,181.0,0.14,2024,1,1,7,10,0,659.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10620,10620,2024-07-24 17:40:00,1.0,0.0,961.30,25.22,100.00,91.25,5.0,181.0,0.00,2024,7,205,17,40,0,91.25
10621,10621,2024-07-24 17:50:00,1.0,0.0,961.06,24.91,100.00,54.00,5.0,181.0,0.00,2024,7,205,17,50,0,54.00
10622,10622,2024-07-24 18:00:00,1.0,0.0,960.91,24.76,100.00,37.25,5.0,181.0,0.00,2024,7,205,18,0,0,37.25
10623,10623,2024-07-24 18:10:00,1.0,0.0,959.86,24.97,100.00,40.33,5.0,181.0,0.00,2024,7,205,18,10,0,40.33


,WindSpeed(m/s),Pressure(hpa),Temperature(°C),Humidity(%),Power(mW),year,month,day,hour,minute,SunlightPred
1158,2.29,1017.71,17.41,90.21,0.00,2024,1,30,6,20,58.93
5861,0.00,1004.82,30.85,100.00,0.81,2024,6,152,17,10,1878.17
3731,0.00,1008.27,27.25,69.07,51.61,2024,3,90,9,30,15091.58
6142,0.00,1006.55,29.05,74.22,0.63,2024,6,156,16,0,1597.25
4267,0.00,1010.51,24.90,100.00,1.32,2024,5,122,9,30,2510.08
...,...,...,...,...,...,...,...,...,...,...,...
5226,0.00,1005.79,24.74,100.00,0.81,2024,5,142,6,0,1815.33
5390,0.00,1006.46,25.80,100.00,0.77,2024,5,144,6,0,1767.42
860,0.00,1025.41,13.74,65.97,0.07,2024,1,24,7,10,612.00
7603,0.00,1005.16,29.80,90.47,3.45,2024,7,195,7,30,3766.25


In [23]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, verbose=0)
# rf_model = XGBRegressor(n_estimators=100, random_state=42)
# rf_model = LGBMRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# 預測測試集的發電量
y_pred = rf_model.predict(X_val)

# 評估模型：計算RMSE
mse = mean_squared_error(y_val, y_pred)
mae = mean_absolute_error(y_val, y_pred)
print(f"MSE: {mse}")
print(f"MAE: {mae}")

# 顯示模型的預測結果
print("預測發電量:", y_pred)
print("實際發電量:", y_val.values)

# 顯示特徵重要性
print("特徵重要性:", rf_model.feature_importances_)

MSE: 25548.953358462622
MAE: 68.70911317978248
預測發電量: [  5.1932   0.682   19.8711 ... 387.444  125.0853  80.9567]
實際發電量: [  4.46   0.77  19.16 ... 370.89 115.97   8.11]
特徵重要性: [0.04590281 0.05406918 0.04117779 0.11298209 0.25714673 0.
 0.00463341 0.0484488  0.09742486 0.00566846 0.33254587]


In [24]:
sum(y_pred<0)

324

In [98]:
np.set_printoptions(suppress=True)
pred = np.round(rf_model.predict(X_test), 2)

In [99]:
y = np.array([1277.55,1369.8,  1462.67, 1540.4,  1594.3,  1631.14, 1659.6,  1671.31, 1674.38,
 1669.7,  1689.69, 1708.54, 1710.23, 1734.19, 1727.19, 1760.06, 1766.71, 1728.95,
 1705.11, 1703.99, 1733.1,  1664.08, 1791.86, 1791.78, 1329.92,  417.97,  291.34,
  195.89,  160.57,  122.99,  105.78,   81.01,   56.43,   36.42,   26.28,   22.13,
   22.02,   19.7,    15.99,   14.8,    12.8,     9.86,    6.89,    4.8,     3.92,
    3.5,     1.72,    0.58])

mean_absolute_error(pred, y)

148.87354166666665

In [ ]:
# for k, v in output.items():
#         if 0 not in v:
#             print(k)
        
#     with open("xtest_num.txt", 'w') as file:
#         file.write(json.dumps(output))

# 156
# 157
# 183
# 189
# 191
# 192